In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q git+https://github.com/tensorflow/examples.git

import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as img
import torch.nn.functional as F
from glob import glob
from sklearn.model_selection import train_test_split
from keras.utils import Sequence
from keras.utils import to_categorical
from skimage.transform import resize
from skimage import img_as_float
from tensorflow_examples.models.pix2pix import pix2pix

In [ ]:
class GeradorSequencias(Sequence):

    def __init__(self, entradas, saidas, batch_size):
        self.entradas = entradas
        self.saidas = saidas
        self.batch_size  = batch_size

    #como calcular a quantidade de lotes: (tamanho do dataset / tamanho do lote)
    def __len__(self):
        return len(self.entradas) // (self.batch_size) # "//" pega o inteiro

    #forma de acessar o índice do lote ([0, 1]) --> para obter o índice 0, multiplique o nº do lote pelo seu tamanho
    def __getitem__(self, id):
        esquerda = id * self.batch_size
        direita = min(esquerda + self.batch_size, len(self.entradas))
        batch_entradas = self.entradas[esquerda:direita]
        batch_saidas = self.saidas[esquerda:direita]

        #redimensionar as imagens de entrada (já está normalizada)
        entradas_redimensionadas = []
        for caminho_img in batch_entradas:
            img_entrada = img.imread(caminho_img + 'image.tif')
            img_redimensionada_entrada = resize(img_entrada, (256, 256))
            entradas_redimensionadas.append(img_redimensionada_entrada)

        #redimensionar as máscaras de saída
        saidas_redimensionadas = []
        for caminho_img in batch_saidas:
            img_saida = img.imread(caminho_img + 'mask.png')
            img_redimensionada_saida = resize(img_saida, (256, 256))
            img_redimensionada_saida = np.array(img_redimensionada_saida)
            min_val = np.min(img_redimensionada_saida)
            max_val = np.max(img_redimensionada_saida)
            img_redimensionada_saida = (img_redimensionada_saida - min_val) / (max_val - min_val)
            threshold = 0.5
            img_redimensionada_saida = tf.where(img_redimensionada_saida < threshold, 0.0, 1.0)
            saidas_redimensionadas.append(img_redimensionada_saida)

        return np.array(entradas_redimensionadas), np.array(saidas_redimensionadas)

In [ ]:
#atribuição dos dados de satétite, contendo talhões e suas máscaras
caminhos = glob('/content/drive/Shareddrives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*/')
np.random.shuffle(caminhos)

#dividindo os caminhos em conjuntos de treino e teste
caminhos_treino, caminhos_teste = train_test_split(caminhos, test_size=0.3, random_state=42)
print("Número de caminhos de treino:", len(caminhos_treino))
print("Número de caminhos de treino:", len(caminhos_teste))


In [ ]:
#variável que contem o X_treino e Y_treino (saída do __getitem__)
sequencia_treino = GeradorSequencias(caminhos_treino, caminhos_treino, 128)

In [ ]:
#variável que contem o X_teste e Y_teste (saída do __getitem__)
sequencia_teste = GeradorSequencias(caminhos_teste, caminhos_teste, 128)

In [ ]:
#verificação da normalização da imagem tif (1ª parte) e da máscara binária (2ª parte)
sequencia_teste.__getitem__(0)

In [ ]:
# Obtendo o primeiro lote de entradas e saídas
primeiro_lote_entradas, primeiro_lote_saidas = sequencia_teste.__getitem__(0)

# Iterando sobre cada entrada e saída no lote
for i in range(len(primeiro_lote_entradas)):
    # Convertendo a entrada e saída para imagens
    img_entrada = primeiro_lote_entradas[i]
    img_saida = primeiro_lote_saidas[i]

    # Mostrando as imagens
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(img_entrada)
    plt.title('Entrada')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(img_saida, cmap='gray')
    plt.title('Saída')
    plt.axis('off')

    plt.show()

In [ ]:
#encoder - modelo MobileNetV2 pré-treinado
base_model = tf.keras.applications.MobileNetV2(input_shape=[256, 256, 3], include_top=False)

# Usar as ativações das camadas
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

# Criar modelo de feature de extração
down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

down_stack.trainable = False

# Decoder/upsampler - série de blocos de upsample implementados nos exemplos do TensorFlow:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def UNet(output_channels:int):
  inputs = tf.keras.layers.Input(shape=[256, 256, 3])

  # Downsampling
  skips = down_stack(inputs)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling e conexões
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # Última camada do modelo
  last = tf.keras.layers.Conv2DTranspose(
      filters=output_channels, kernel_size=3, strides=2,
      padding='same')  #64x64 -> 256x256

  x = last(x)

  # Ativação sigmoid
  x = tf.keras.layers.Activation('sigmoid')(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
with tf.device('/gpu:0'):
    OUTPUT_CLASSES = 1
    model = UNet(output_channels=OUTPUT_CLASSES)
    model.compile(optimizer='adam',
                loss="binary_crossentropy",
                metrics=['accuracy'])

    tf.keras.utils.plot_model(model, show_shapes=True)

    EPOCHS = 100
    BATCH_SIZE = 128

    H = model.fit(sequencia_treino, epochs=EPOCHS,
                            validation_data=sequencia_teste)

    # Avaliar o modelo
    loss, accuracy = model.evaluate(sequencia_teste)
    print(f'Acurácia do modelo: {accuracy}')

    # Mostrar o resultado
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(H.epoch, H.history["loss"], label="train_loss")
    plt.plot(H.epoch, H.history["val_loss"], label="val_loss")
    plt.plot(H.epoch, H.history["accuracy"], label="train_acc")
    plt.plot(H.epoch, H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.show()


In [ ]:
with tf.device('/gpu:0'):
    # Gerar as saídas do modelo para um conjunto de entradas de teste
    saidas_modelo = model.predict(sequencia_teste)

    # Iterar sobre cada saída do modelo
    for i in range(BATCH_SIZE):
        # Obter a entrada correspondente e a saída real
        img_entrada = primeiro_lote_entradas[i]
        img_saida_real = primeiro_lote_saidas[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_modelo[i]

        #Aplicar limiarização apenas durante a inferência
        img_saida_modelo_limiarizada = np.where(img_saida_modelo < 0.5, 0, 1)

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada)
        plt.title('Entrada')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real, cmap='gray')
        plt.title('Saída Esperada')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo_limiarizada, cmap='gray')
        plt.title('Saída do Modelo (Limiarizada)')
        plt.axis('off')

        plt.show()